# Import libraries

# Define functions for calculating GLCM features for all images
Place holder path_to_convoluted variable with the correct path to convoluted images.

In [1]:
import os
import numpy as np
from PIL import Image
import multiprocessing
import pycwt
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
from skimage import io, color
import pandas as pd
import pickle
import cv2
import matplotlib.pyplot as plt 
def read_image_files_from_directory(directory):
    image_files = []
    supported_formats = (".jpg", ".jpeg", ".png", ".gif", ".bmp")  # Add or remove formats as needed

    for file in os.listdir(directory):
        if file.lower().endswith(supported_formats):
            image_path = os.path.join(directory, file)
            image_files.append(image_path)

    return image_files

def extract_glcm(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True):
    # Convert image to grayscale
    gray_image = color.rgb2gray(image)

    # Convert grayscale image to integer values
    gray_image = (gray_image * (levels - 1)).astype(np.uint8)

    # Compute GLCM
    glcm = graycomatrix(gray_image, distances, angles, levels=levels,
                        symmetric=symmetric, normed=normed)

    return glcm
def plot_img(image_array):
    num_images = len(image_array)
    rows = 2
    cols = 2

    # Create a figure and axes
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(10, 6))

    # Flatten the axes array for easier indexing
    axes = axes.flatten()

    # Generate subplots dynamically
    for i in range(num_images):
        # Plot image on each subplot
        axes[i].imshow(image_array[i], cmap='gray')
        axes[i].set_title(f'Image {i+1}')
        axes[i].axis('off')

    # Hide any extra subplots if necessary
    for j in range(num_images, rows*cols):
        axes[j].axis('off')

    # Adjust spacing between subplots
    plt.tight_layout()

    # Show the plot
    plt.show()
    
path_to_convoluted = '/home/rajdeep/projects/image_segmentation/data/convoluted/'

# Calculate GLCM features
For this we will be reading all the convoluted images in convoluted directory.

In [2]:
glcm_img = {'img':[],'contrast':[],'correlation':[],'energy':[],\
            'dissimilarity':[],'ASM':[],'homogeneity':[]}
df = pd.read_csv('./GroundTruth.csv')
columns = ['image','MEL','NV','BCC','AKIEC','BKL','DF','VASC','contrast'\
           ,'correlation','energy','dissimilarity','ASM','homogeneity']
index_dict = {value: index for index, value in enumerate(columns)}
print(index_dict)

df['contrast'] = 0.0
df['correlation'] = 0.0
df['energy'] = 0.0
df['dissimilarity'] = 0.0
df['ASM'] = 0.0
df['homogeneity'] = 0.0
#df = df.iloc[:5,:]
def cal_glcm(x):
    
    image_array = cv2.imread(path_to_convoluted+x['image'].values[0]+'.jpg')
    glcm = extract_glcm(image_array)
    contrast = graycoprops(glcm, 'contrast')
    correlation = graycoprops(glcm, 'correlation')
    energy = graycoprops(glcm, 'energy')
    dissimilarity = graycoprops(glcm,'dissimilarity')
    asm = graycoprops(glcm,'ASM')
    homogeneity = graycoprops(glcm, 'homogeneity')
    x['contrast'] = contrast
    x['correlation'] = correlation
    x['energy'] = energy
    x['dissimilarity'] = dissimilarity
    x['ASM'] = asm
    x['homogeneity'] = homogeneity
    return x
res = None

num_chunks = len(df)
chunks = np.array_split(df, num_chunks)
res = None
with multiprocessing.Pool(processes=32) as pool:
    res = pool.map(cal_glcm,chunks)
    
pool.close()
pool.join()
    
df = pd.concat(res)
print(res)

    
    


{'image': 0, 'MEL': 1, 'NV': 2, 'BCC': 3, 'AKIEC': 4, 'BKL': 5, 'DF': 6, 'VASC': 7, 'contrast': 8, 'correlation': 9, 'energy': 10, 'dissimilarity': 11, 'ASM': 12, 'homogeneity': 13}


[ WARN:0@2.892] global loadsave.cpp:248 findDecoder imread_('/home/rajdeep/projects/image_segmentation/data/convoluted/ISIC_0024385.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2.898] global loadsave.cpp:248 findDecoder imread_('/home/rajdeep/projects/image_segmentation/data/convoluted/ISIC_0024543.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2.901] global loadsave.cpp:248 findDecoder imread_('/home/rajdeep/projects/image_segmentation/data/convoluted/ISIC_0024622.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2.897] global loadsave.cpp:248 findDecoder imread_('/home/rajdeep/projects/image_segmentation/data/convoluted/ISIC_0024464.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2.904] global loadsave.cpp:248 findDecoder imread_('/home/rajdeep/projects/image_segmentation/data/convoluted/ISIC_0024780.jpg'): can't open/read file: check file path/integrity
[ WARN:0@2.909] global loadsave.cpp:248 findDecoder imread_('/hom

[ WARN:0@3.216] global loadsave.cpp:248 findDecoder imread_('/home/rajdeep/projects/image_segmentation/data/convoluted/ISIC_0029994.jpg'): can't open/read file: check file path/integrity
[ WARN:0@3.216] global loadsave.cpp:248 findDecoder imread_('/home/rajdeep/projects/image_segmentation/data/convoluted/ISIC_0029915.jpg'): can't open/read file: check file path/integrity
[ WARN:0@3.222] global loadsave.cpp:248 findDecoder imread_('/home/rajdeep/projects/image_segmentation/data/convoluted/ISIC_0030073.jpg'): can't open/read file: check file path/integrity
[ WARN:0@3.224] global loadsave.cpp:248 findDecoder imread_('/home/rajdeep/projects/image_segmentation/data/convoluted/ISIC_0030152.jpg'): can't open/read file: check file path/integrity
[ WARN:0@3.225] global loadsave.cpp:248 findDecoder imread_('/home/rajdeep/projects/image_segmentation/data/convoluted/ISIC_0030231.jpg'): can't open/read file: check file path/integrity
[ WARN:0@3.228] global loadsave.cpp:248 findDecoder imread_('/hom

IndexError: tuple index out of range

In [4]:
def preprocess_category_name(x):
    #print(x)
    if x['MEL']==1.0:
        return 'MEL'
    if x['NV']==1.0:
        return 'NV'
    if x['BCC']==1.0:
        return 'BCC'
    if x['AKIEC']==1.0:
        return 'AKIEC'
    if x['BKL']==1.0:
        return 'BKL'
    if x['DF']==1.0:
        return 'DF'
    if x['VASC']==1.0:
        return 'VASC'
    
def preprocess_category_int(x):
    #print(x)
    if x['MEL']==1.0:
        return 0
    if x['NV']==1.0:
        return 1
    if x['BCC']==1.0:
        return 2
    if x['AKIEC']==1.0:
        return 3
    if x['BKL']==1.0:
        return 4
    if x['DF']==1.0:
        return 5
    if x['VASC']==1.0:
        return 6
df['class_name'] = df.apply(lambda x:preprocess_category_name(x),axis=1)
df['class'] = df.apply(lambda x:preprocess_category_int(x),axis=1)
df.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC,contrast,correlation,energy,dissimilarity,ASM,homogeneity,class_name,class
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,1
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,1
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,1
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,1
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL,0


# Save features to CSV

In [5]:
df.to_csv('./glcm_features.csv',index=False)